# 2.3 LLM Generation Parameters

## Playground Notebook

Generation parameters are the **knobs and dials** that control how a model produces text. They don't change *what* the model knows — they change *how* it samples from its knowledge.

| Parameter | What It Controls |
|-----------|------------------|
| **Temperature** | Randomness and creativity in outputs |
| **Top-P** | Limits token selection to a cumulative probability threshold |
| **Top-K** | Limits sampling to only the K most likely tokens |
| **Max Tokens** | Maximum number of tokens the model can generate |
| **Frequency Penalty** | Penalizes tokens based on how often they've appeared |
| **Presence Penalty** | Penalizes any token that has appeared at least once |
| **Stop Sequences** | Strings that immediately halt generation |

---

In [1]:
import json
import time
from IPython.display import display, Markdown, HTML
from langchain_ollama import ChatOllama
from langchain_core.messages import SystemMessage, HumanMessage

# ============================================================
#  CONFIGURATION - Change the model name here if needed
# ============================================================
MODEL = "qwen2.5:1.5b"  # Options: "qwen2.5:1.5b", "llama3.2", "mistral", "gemma2", etc.

c:\Users\shiva\AppData\Local\Programs\Python\Python314\Lib\site-packages\langchain_core\_api\deprecation.py:25: UserWarning: Core Pydantic V1 functionality isn't compatible with Python 3.14 or greater.
  from pydantic.v1.fields import FieldInfo as FieldInfoV1


In [2]:
# ============================================================
#  HELPER FUNCTIONS
# ============================================================

def generate(prompt, system="You are a helpful assistant.", **kwargs):
    """Send a prompt with custom generation parameters and return the response."""
    llm = ChatOllama(model=MODEL, **kwargs)
    messages = [SystemMessage(content=system), HumanMessage(content=prompt)]
    start = time.time()
    response = llm.invoke(messages)
    elapsed = time.time() - start
    content = response.content
    display(Markdown(content))
    print(f"\n⏱️ {elapsed:.2f}s | {len(content)} chars")
    return content


def compare(prompt, configs, system="You are a helpful assistant."):
    """Run the same prompt with different parameter configs side by side."""
    results = {}
    for cfg in configs:
        label = cfg.pop("label")
        print(f"\n{'=' * 60}")
        print(f"  {label}")
        params_str = ', '.join(f'{k}={v}' for k, v in cfg.items())
        print(f"  Parameters: {params_str}")
        print(f"{'=' * 60}")
        results[label] = generate(prompt, system=system, **cfg)
        cfg["label"] = label  # restore label
    return results


print(f"✅ Using model: {MODEL}")

✅ Using model: qwen2.5:1.5b


---

## 1. Temperature — Controlling Randomness

Temperature scales the probability distribution before sampling:

```
Temperature = 0.0  →  Always pick the most likely token (deterministic)
Temperature = 0.7  →  Balanced creativity (good default)
Temperature = 1.5  →  Very random, creative, sometimes incoherent
```

**Think of it like a dial:**
```
FOCUSED ◄──────────────────────► CREATIVE
  0.0       0.3    0.7    1.0      1.5+
```

### Experiment 1A: Low vs. Medium vs. High Temperature

In [4]:
prompt = "Write a one-sentence description of the ocean."

configs = [
    {"label": "🧊 Temperature = 0.0 (Deterministic)", "temperature": 0.0},
    {"label": "⚖️ Temperature = 0.7 (Balanced)",     "temperature": 0.7},
    {"label": "🔥 Temperature = 1.5 (Very Creative)", "temperature": 1.5},
]

_ = compare(prompt, configs)


  🧊 Temperature = 0.0 (Deterministic)
  Parameters: temperature=0.0


The vast and mysterious ocean covers approximately 71% of Earth's surface and is home to an incredible array of life forms, from tiny plankton to massive whales.


⏱️ 1.90s | 161 chars

  ⚖️ Temperature = 0.7 (Balanced)
  Parameters: temperature=0.7


The vast and mysterious ocean covers more than 70% of our planet's surface and is home to an incredible array of life forms.


⏱️ 0.69s | 124 chars

  🔥 Temperature = 1.5 (Very Creative)
  Parameters: temperature=1.5


The vast,蔚蓝、波澜壮阔的海洋，是地球上生命的重要源泉。


⏱️ 0.58s | 32 chars


### Experiment 1B: Temperature & Consistency

At **temperature 0**, the model should produce the *same* output every time. At higher temperatures, each run differs. Let's verify.

In [5]:
prompt = "Name one color."
num_runs = 4

for temp in [0.0, 1.0]:
    print(f"\n{'=' * 60}")
    print(f"  Temperature = {temp} — Running {num_runs} times")
    print(f"{'=' * 60}")
    outputs = []
    for i in range(num_runs):
        llm = ChatOllama(model=MODEL, temperature=temp)
        resp = llm.invoke([HumanMessage(content=prompt)])
        text = resp.content.strip()
        outputs.append(text)
        print(f"  Run {i+1}: {text[:80]}")
    unique = len(set(outputs))
    print(f"  → Unique outputs: {unique}/{num_runs}")


  Temperature = 0.0 — Running 4 times
  Run 1: Blue is one of the colors.
  Run 2: Blue is one of the colors.
  Run 3: Blue is one of the colors.
  Run 4: Blue is one of the colors.
  → Unique outputs: 1/4

  Temperature = 1.0 — Running 4 times
  Run 1: I'll choose blue as the color to name.
  Run 2: Sure! Here's a random choice: **Blue**. Blue is the color associated with calmne
  Run 3: Blue is one of the colors.
  Run 4: Blue is a color.
  → Unique outputs: 4/4


### Experiment 1C: Temperature for Different Tasks

Different tasks need different temperature settings.

In [ ]:
tasks = [
    {
        "label": "Factual Q&A (low temp is better)",
        "prompt": "What is the capital of Inida? Answer in one sentence.",
        "temps": [0.0, 0.7, 1.5]
    },
    {
        "label": "Creative Writing (higher temp is better)",
        "prompt": "Describe a sunset without using the word 'sun' or 'sky'. One sentence only.",
        "temps": [0.0, 0.7, 1.5]
    },
    {
        "label": "Code Generation (low temp is better)",
        "prompt": "Write a Python one-liner that reverses a string.",
        "temps": [0.0, 0.7, 1.5]
    }
]

for task in tasks:
    print(f"\n{'#' * 60}")
    print(f"  TASK: {task['label']}")
    print(f"{'#' * 60}")
    for temp in task["temps"]:
        print(f"\n--- Temperature = {temp} ---")
        _ = generate(task["prompt"], temperature=temp)

    print(f"\n💡 Observation: Compare how temperature affects accuracy vs. creativity above.")



############################################################
  TASK: Factual Q&A (low temp is better)
############################################################

--- Temperature = 0.0 ---


The boiling point of water at standard atmospheric pressure (1 atmosphere) is 100 degrees Celsius.


⏱️ 0.69s | 98 chars

--- Temperature = 0.7 ---


The boiling point of water at standard atmospheric pressure (1 atmosphere) is 100 degrees Celsius.


⏱️ 0.60s | 98 chars

--- Temperature = 1.5 ---


...


⏱️ 4.90s | 333 chars

--- Temperature = 0.7 ---


Here's a one-liner in Python to reverse a string:

```python
reversed_string = ''.join(reversed(string))
```

This line of code takes the input string, `string`, and uses the built-in `reversed()` function to reverse it character by character. The result is then joined back into a single string using the `join()` method with an empty string as the separator.

Note that this one-liner only works if you're working in Python 3.x where strings are Unicode, or if you want to explicitly cast the input to a string type (e.g., `str(input())`).


⏱️ 9.47s | 541 chars

--- Temperature = 1.5 ---


Here is a one-line Python solution using slicing:

```python
reversed_string = s[::-1]
```

However, note that the slicing syntax `[::-1]` creates a copy of the original string in reverse order, so this is just a view and not an actual change to the original string. If you want to modify the original string, you can assign it back to itself:

```python
s = "Hello, World!"
reversed_string = s[::-1]
```

Alternatively, you could use Python's `reversed` function directly on the string:

```python
reversed_string = "".join(reversed("Hello, World!"))
```

However, note that this method also returns a new reversed version of the original string, so it is only for assignment and not modification.


⏱️ 15.98s | 698 chars

💡 Observation: Compare how temperature affects accuracy vs. creativity above.


---

## 2. Top-P (Nucleus Sampling) — Probability Threshold

Top-P limits the model to the **smallest set of tokens** whose cumulative probability adds up to P.

```
Top-P = 0.1  →  Only the top ~10% probability mass (very focused)
Top-P = 0.9  →  Top ~90% probability mass (more diverse)
Top-P = 1.0  →  Consider all tokens (no filtering)
```

**Example — Next token probabilities:**
```
Token:   "the"   "a"    "an"   "one"  "my"  "some" ...
Prob:     0.35   0.25   0.15   0.10   0.05   0.03  ...

Top-P=0.5 → selects {"the", "a"} (0.35+0.25=0.60 ≥ 0.5)
Top-P=0.9 → selects {"the", "a", "an", "one", "my"}
```

### Experiment 2A: Top-P Narrow vs. Wide

In [8]:
prompt = "List 5 unusual hobbies someone might enjoy."

configs = [
    {"label": "🎯 Top-P = 0.1 (Very Focused)",  "top_p": 0.1, "temperature": 0.8},
    {"label": "⚖️ Top-P = 0.5 (Moderate)",       "top_p": 0.5, "temperature": 0.8},
    {"label": "🌊 Top-P = 0.95 (Diverse)",       "top_p": 0.95, "temperature": 0.8},
]

_ = compare(prompt, configs)


  🎯 Top-P = 0.1 (Very Focused)
  Parameters: top_p=0.1, temperature=0.8


1. Playing the banjo: This instrument is often associated with country music and bluegrass, but it can also be played as an unusual hobby.
2. Collecting rare stamps: Many people collect stamps for fun or to add to their collections of memorabilia.
3. Growing a garden: While many hobbies involve growing plants, some individuals enjoy the challenge of growing unique or exotic species that are not commonly found in local gardens.
4. Playing chess: Chess is often considered a boring hobby, but it can be enjoyed by those who find it challenging and intellectually stimulating.
5. Collecting vintage toys: Some people collect old toys as a way to preserve history and add nostalgia to their homes.


⏱️ 3.30s | 697 chars

  ⚖️ Top-P = 0.5 (Moderate)
  Parameters: top_p=0.5, temperature=0.8


1. Playing with slime: Some people find it fun to create and play with slimy substances.
2. Rock climbing: This adventurous hobby involves scaling steep rock faces, often outdoors or in indoor climbing walls.
3. Making jewelry from recycled materials: Instead of using new metals and gemstones, some individuals collect old items like broken glass, buttons, and metal scraps and transform them into beautiful jewelry pieces.
4. Growing a garden with unusual plants: Some people enjoy growing rare or exotic plants that are not commonly found in their area.
5. Building model airplanes: This hobby involves creating miniature versions of real aircraft using plastic, wood, and other materials.


⏱️ 2.18s | 692 chars

  🌊 Top-P = 0.95 (Diverse)
  Parameters: top_p=0.95, temperature=0.8


1. Collecting rare stamps or coins.
2. Playing Dungeons and Dragons or other tabletop role-playing games.
3. Learning to juggle or perform acrobatics for entertainment.
4. Growing exotic houseplants that require specific care and attention.
5. Knitting or crocheting clothes, accessories, or blankets that are both functional and beautiful.


⏱️ 1.40s | 340 chars


### Experiment 2B: Temperature vs. Top-P — They Work Together

Temperature reshapes probabilities *first*, then Top-P filters the result. Using both together gives fine-grained control.

In [9]:
prompt = "Invent a name for a fantasy tavern."

configs = [
    {"label": "Low Temp + Low Top-P (Most predictable)",   "temperature": 0.2, "top_p": 0.3},
    {"label": "Low Temp + High Top-P",                     "temperature": 0.2, "top_p": 0.95},
    {"label": "High Temp + Low Top-P",                     "temperature": 1.2, "top_p": 0.3},
    {"label": "High Temp + High Top-P (Most creative)",    "temperature": 1.2, "top_p": 0.95},
]

_ = compare(prompt, configs)


  Low Temp + Low Top-P (Most predictable)
  Parameters: temperature=0.2, top_p=0.3


"Drinking Springs"


⏱️ 0.49s | 18 chars

  Low Temp + High Top-P
  Parameters: temperature=0.2, top_p=0.95


"Feastfire's Forge"


⏱️ 0.30s | 19 chars

  High Temp + Low Top-P
  Parameters: temperature=1.2, top_p=0.3


"Druid's Den"


⏱️ 0.20s | 13 chars

  High Temp + High Top-P (Most creative)
  Parameters: temperature=1.2, top_p=0.95


"The Shadowy Vale"


⏱️ 0.20s | 18 chars


---

## 3. Top-K — Fixed Token Pool Size

Top-K is simpler than Top-P: it always considers exactly the **K most likely tokens**, regardless of their probabilities.

```
Top-K = 1   →  Greedy decoding (always pick the #1 token)
Top-K = 10  →  Choose from top 10 tokens
Top-K = 50  →  Choose from top 50 tokens (more variety)
```

### Experiment 3A: Top-K Values Compared

In [12]:
prompt = "Give me a one-word synonym for 'Work'."

configs = [
    {"label": "Top-K = 1 (Greedy — always picks top token)",  "top_k": 1,  "temperature": 0.8},
    {"label": "Top-K = 5",                                     "top_k": 5,  "temperature": 0.8},
    {"label": "Top-K = 40 (Default for many models)",          "top_k": 40, "temperature": 0.8},
    {"label": "Top-K = 100 (Wide pool)",                       "top_k": 100, "temperature": 0.8},
]

_ = compare(prompt, configs)


  Top-K = 1 (Greedy — always picks top token)
  Parameters: top_k=1, temperature=0.8


Job


⏱️ 0.77s | 3 chars

  Top-K = 5
  Parameters: top_k=5, temperature=0.8


Task.


⏱️ 3.22s | 5 chars

  Top-K = 40 (Default for many models)
  Parameters: top_k=40, temperature=0.8


Job


⏱️ 3.42s | 3 chars

  Top-K = 100 (Wide pool)
  Parameters: top_k=100, temperature=0.8


Job


⏱️ 3.02s | 3 chars


### Experiment 3B: Top-K Consistency Test

With `top_k=1`, the output should be identical every time (greedy). Let's check.

In [13]:
prompt = "What is 2 + 2? Reply with just the number."
num_runs = 4

for k_val in [1, 50]:
    print(f"\n{'=' * 60}")
    print(f"  Top-K = {k_val} — Running {num_runs} times")
    print(f"{'=' * 60}")
    outputs = []
    for i in range(num_runs):
        llm = ChatOllama(model=MODEL, top_k=k_val, temperature=0.8)
        resp = llm.invoke([HumanMessage(content=prompt)])
        text = resp.content.strip()
        outputs.append(text)
        print(f"  Run {i+1}: {text[:80]}")
    unique = len(set(outputs))
    print(f"  → Unique outputs: {unique}/{num_runs}")


  Top-K = 1 — Running 4 times
  Run 1: 4
  Run 2: 4
  Run 3: 4
  Run 4: 4
  → Unique outputs: 1/4

  Top-K = 50 — Running 4 times
  Run 1: 4
  Run 2: 4
  Run 3: 4
  Run 4: 4
  → Unique outputs: 1/4


### Top-K vs. Top-P — When to Use Which?

| Feature | Top-K | Top-P |
|---------|-------|-------|
| Pool size | **Fixed** (always K tokens) | **Dynamic** (varies per step) |
| Adapts to confidence? | No | Yes |
| Best for | Simple control | Nuanced generation |
| Common defaults | K=40 | P=0.9 |

---

## 4. Max Tokens — Controlling Response Length

**Max Tokens** (called `num_predict` in Ollama) sets a hard ceiling on how many tokens the model generates. It does NOT guarantee that length — the model may stop earlier if it finishes its thought.

```
1 token ≈ 4 characters ≈ ¾ of a word (English)
```

### Experiment 4A: Varying Max Tokens

In [14]:
prompt = "Explain the theory of relativity in detail."

configs = [
    {"label": "🔹 Max Tokens = 20 (Very Short)",   "num_predict": 20},
    {"label": "🔹 Max Tokens = 80 (Short)",        "num_predict": 80},
    {"label": "🔹 Max Tokens = 300 (Medium)",      "num_predict": 300},
]

_ = compare(prompt, configs)


  🔹 Max Tokens = 20 (Very Short)
  Parameters: num_predict=20


The theory of relativity is a physical theory that describes how space and time relate to matter, according


⏱️ 1.47s | 107 chars

  🔹 Max Tokens = 80 (Short)
  Parameters: num_predict=80


The theory of relativity is a set of scientific theories developed by two physicists, Albert Einstein and Hendrik Lorentz, that describe how space and time relate to each other. The theory was first published in 1905 and has since become one of the most important theoretical frameworks in modern physics.

One of the key ideas behind the theory is that space and time are not absolute quantities -


⏱️ 4.73s | 398 chars

  🔹 Max Tokens = 300 (Medium)
  Parameters: num_predict=300


The theory of relativity is a set of scientific theories that describes how space and time affect objects moving at different speeds or in different gravitational fields, including acceleration. It was developed by two physicists, Albert Einstein, first as special relativity (1905) and then general relativity (1915).

Special Relativity:
The theory of special relativity introduced the concepts of spacetime and the principle of relativity to account for how the laws of physics apply in all inertial frames. It describes the effects that would occur if objects were observed moving at high speeds or near a strong gravitational field.

Key ideas in special relativity include:

1. The speed of light is constant, regardless of the motion of the observer.
2. Time dilation: time slows down for an object moving at high speeds compared to observers on Earth.
3. Length contraction: objects appear shorter than their length when they are moving fast relative to a stationary observer.
4. Mass-energy equivalence: energy and mass can be converted into one another, as described by the equation E=mc^2.

General Relativity:
The theory of general relativity describes gravity as the curvature in spacetime caused by other massive objects occupying spacetime. This means that massive bodies warp spacetime, causing them to attract each other gravitationally because they are following geodesics through this curved space.

Key ideas in general relativity include:

1. Gravitational lensing: light bends around massive objects like stars or galaxies


⏱️ 23.97s | 1544 chars


### Experiment 4B: Max Tokens — Cutting Off Mid-Sentence

Watch what happens when the limit is too low — the model gets cut off mid-thought.

In [ ]:
prompt = "Tell me a short story about a brave knight."

for max_tok in [10, 30, 100]:
    print(f"\n{'=' * 60}")
    print(f"  num_predict = {max_tok}")
    print(f"{'=' * 60}")
    _ = generate(prompt, num_predict=max_tok)

print("\n💡 Notice how low limits produce incomplete responses!")

---

## 5. Frequency Penalty — Reducing Repetition

Frequency Penalty penalizes tokens **proportionally to how many times** they've already appeared in the output. The more a word repeats, the harder it gets penalized.

```
Penalty = 0.0  →  No penalty (default)
Penalty > 0    →  Discourages repetition (higher = stronger)
Penalty < 0    →  Encourages repetition (rarely useful)
```

In Ollama, this maps to the `repeat_penalty` parameter (default 1.1; values > 1.0 penalize repetition).

### Experiment 5A: Repetition With and Without Penalty

In [ ]:
# A prompt that tends to cause repetitive output
prompt = "Write the word 'hello' in 10 different creative ways."

configs = [
    {"label": "🔁 repeat_penalty = 1.0 (No penalty)",       "repeat_penalty": 1.0},
    {"label": "⚖️ repeat_penalty = 1.1 (Default / Mild)",   "repeat_penalty": 1.1},
    {"label": "🚫 repeat_penalty = 1.5 (Strong penalty)",   "repeat_penalty": 1.5},
]

_ = compare(prompt, configs)

### Experiment 5B: Frequency Penalty on Longer Text

Repetition is more visible in longer outputs. Let's test with a paragraph-length prompt.

In [ ]:
prompt = "Write a paragraph about the importance of reading books. Aim for about 100 words."

for penalty in [1.0, 1.3]:
    print(f"\n{'=' * 60}")
    print(f"  repeat_penalty = {penalty}")
    print(f"{'=' * 60}")
    result = generate(prompt, repeat_penalty=penalty, num_predict=200)

    # Count word frequency to show repetition
    words = result.lower().split()
    word_counts = {}
    for w in words:
        w_clean = w.strip('.,!?;:')
        if len(w_clean) > 3:  # skip short words
            word_counts[w_clean] = word_counts.get(w_clean, 0) + 1
    repeated = {w: c for w, c in word_counts.items() if c >= 3}
    if repeated:
        print(f"  📊 Words repeated 3+ times: {repeated}")
    else:
        print(f"  📊 No words repeated 3+ times — good variety!")

---

## 6. Presence Penalty — Encouraging Topic Diversity

Unlike Frequency Penalty (which scales with count), Presence Penalty applies a **flat penalty** to any token that has appeared **at least once**. It doesn't matter if it appeared 1 time or 50 — the penalty is the same.

```
Frequency Penalty:  "the" appeared 5x → penalized 5× as much
Presence  Penalty:  "the" appeared 5x → same penalty as if it appeared 1x
```

This encourages the model to bring in **new topics and vocabulary** rather than just avoiding repetition.

> **Note:** In Ollama, `repeat_penalty` combines both frequency and presence penalty effects. We simulate the distinction below.

### Experiment 6A: Presence Penalty Effect on Vocabulary Diversity

In [16]:
prompt = "List 10 different animals. Just the names, one per line."

configs = [
    {"label": "repeat_penalty = 1.0 (No penalty)",    "repeat_penalty": 1.0, "temperature": 0.7},
    {"label": "repeat_penalty = 1.2 (Moderate)",      "repeat_penalty": 1.2, "temperature": 0.7},
    {"label": "repeat_penalty = 1.8 (Aggressive)",    "repeat_penalty": 1.8, "temperature": 0.7},
]

results = compare(prompt, configs)

# Analyze unique words in each
print(f"\n{'=' * 60}")
print("VOCABULARY DIVERSITY ANALYSIS")
print(f"{'=' * 60}")
for label, text in results.items():
    words = set(text.lower().split())
    print(f"  {label[:40]:40s} → {len(words)} unique words")


  repeat_penalty = 1.0 (No penalty)
  Parameters: repeat_penalty=1.0, temperature=0.7


1. Elephant
2. Giraffe
3. Monkey
4. Lion
5. Tiger
6. Penguin
7. Dolphin
8. Fox
9. Kangaroo
10. Octopus


⏱️ 2.38s | 102 chars

  repeat_penalty = 1.2 (Moderate)
  Parameters: repeat_penalty=1.2, temperature=0.7


Elephant  
Lion  
Tiger  
Bear  
Monkey  
Penguin  
Kangaroo  
Giraffe  
Eagle  
Crocodile


⏱️ 2.05s | 90 chars

  repeat_penalty = 1.8 (Aggressive)
  Parameters: repeat_penalty=1.8, temperature=0.7



KeyboardInterrupt



### Frequency vs. Presence Penalty — Comparison

| Aspect | Frequency Penalty | Presence Penalty |
|--------|-------------------|------------------|
| Scales with count? | **Yes** — more repetitions = more penalty | **No** — flat penalty after first use |
| Best for | Reducing word-level repetition | Encouraging topic diversity |
| Use case | Preventing "the the the..." | Making model explore new ideas |

---

## 7. Stop Sequences — Halting Generation

Stop sequences are strings that **immediately end** the model's generation when encountered. The model stops *before* including the stop string in the output.

Common uses:
- Stop at a newline (`\n`) for single-line answers
- Stop at a delimiter (`---`, `END`) for structured extraction
- Stop at a role marker (`User:`) to prevent the model from simulating conversation

### Experiment 7A: Stopping at a Newline (Single-Line Answers)

In [ ]:
prompt = "Name a famous scientist and describe their contribution."

print("=" * 60)
print("WITHOUT stop sequence")
print("=" * 60)
_ = generate(prompt)

print("\n" + "=" * 60)
print("WITH stop=['.'] — stops at first period")
print("=" * 60)
_ = generate(prompt, stop=["."])

### Experiment 7B: Stop Sequences for Structured Output

In [ ]:
prompt = """Extract the person's name from the text below.

Text: "Dr. Sarah Chen published her findings on climate change last Tuesday."

Name:"""

print("=" * 60)
print("WITH stop=['\\n'] — stops after extracting the name")
print("=" * 60)
_ = generate(prompt, stop=["\n"], temperature=0.0)

### Experiment 7C: Stop Sequences to Prevent Role-Playing

In [ ]:
prompt = """Answer the user's question in one sentence.

User: What is gravity?
Assistant:"""

print("=" * 60)
print("WITHOUT stop — model might continue as 'User:' and 'Assistant:'")
print("=" * 60)
_ = generate(prompt, num_predict=200)

print("\n" + "=" * 60)
print("WITH stop=['User:', '\\n\\n'] — halts after one response")
print("=" * 60)
_ = generate(prompt, stop=["User:", "\n\n"], num_predict=200)

---

## 8. Combining Parameters — Real-World Recipes

In practice, you'll combine multiple parameters together. Here are some common "recipes":

| Use Case | Temperature | Top-P | Top-K | Repeat Penalty | Max Tokens |
|----------|-------------|-------|-------|----------------|------------|
| Factual Q&A | 0.0 | 1.0 | 1 | 1.0 | 100-200 |
| Creative Writing | 0.9-1.2 | 0.9 | 50 | 1.2 | 500+ |
| Code Generation | 0.0-0.2 | 0.95 | 40 | 1.1 | 500 |
| Brainstorming | 1.0 | 0.95 | 100 | 1.3 | 300 |
| Data Extraction | 0.0 | 1.0 | 1 | 1.0 | 100 |

### Experiment 8A: Recipe Comparison

In [ ]:
prompt = "Suggest 3 startup ideas related to artificial intelligence."

configs = [
    {
        "label": "📋 Conservative (Factual style)",
        "temperature": 0.1, "top_p": 1.0, "top_k": 1, "repeat_penalty": 1.0, "num_predict": 200
    },
    {
        "label": "⚖️ Balanced (General purpose)",
        "temperature": 0.7, "top_p": 0.9, "top_k": 40, "repeat_penalty": 1.1, "num_predict": 200
    },
    {
        "label": "🚀 Creative (Brainstorming)",
        "temperature": 1.1, "top_p": 0.95, "top_k": 100, "repeat_penalty": 1.3, "num_predict": 200
    }
]

_ = compare(prompt, configs)

### Experiment 8B: Code Generation Recipe

In [ ]:
prompt = "Write a Python function that checks if a string is a palindrome."

system = "You are a Python developer. Write clean, well-commented code. Only output the code, nothing else."

configs = [
    {
        "label": "🎯 Precise Code (temp=0, top_k=1)",
        "temperature": 0.0, "top_k": 1, "num_predict": 300
    },
    {
        "label": "🎨 Creative Code (temp=0.8, top_k=50)",
        "temperature": 0.8, "top_k": 50, "num_predict": 300
    },
]

_ = compare(prompt, configs, system=system)

---

## 9. Sandbox — Try It Yourself!

Experiment with any combination of parameters below.

In [ ]:
# ============================================================
#  SANDBOX - Tweak these values and re-run!
# ============================================================

my_prompt     = "Describe the future of space travel in 3 sentences."
my_system     = "You are a futurist and science communicator."

my_params = {
    "temperature":    0.7,    # 0.0 to 2.0
    "top_p":          0.9,    # 0.0 to 1.0
    "top_k":          40,     # 1 to 100+
    "num_predict":    200,    # max tokens to generate
    "repeat_penalty": 1.1,    # 1.0 = off, higher = less repetition
    # "stop":         ["."],  # uncomment to stop at first period
}

# ============================================================

print("YOUR CUSTOM EXPERIMENT")
print("=" * 60)
params_str = '\n'.join(f"  {k:20s} = {v}" for k, v in my_params.items())
print(params_str)
print("=" * 60)
_ = generate(my_prompt, system=my_system, **my_params)

---

## Key Takeaways

| Parameter | What It Does | Typical Range | When to Adjust |
|-----------|-------------|---------------|----------------|
| **Temperature** | Controls randomness | 0.0 – 1.5 | Lower for facts, higher for creativity |
| **Top-P** | Dynamic probability cutoff | 0.1 – 1.0 | Use ~0.9 for general; lower for precision |
| **Top-K** | Fixed candidate pool size | 1 – 100 | 1 for greedy; 40-50 for balanced |
| **Max Tokens** | Hard output length limit | 10 – 4096 | Match to your expected output length |
| **Frequency Penalty** | Penalizes repeated tokens proportionally | 1.0 – 1.5 | Increase for less repetition |
| **Presence Penalty** | Flat penalty on any used token | 1.0 – 1.5 | Increase for broader vocabulary |
| **Stop Sequences** | Halts generation at specific strings | N/A | Use for structured/single-line output |

### Rules of Thumb

1. **Start with defaults** — temperature=0.7, top_p=0.9, top_k=40
2. **Adjust one parameter at a time** — so you can see what each one does
3. **Temperature and Top-P overlap** — usually tune one or the other, not both aggressively
4. **Low temperature + Top-K=1** — effectively deterministic (greedy decoding)
5. **Stop sequences are underused** — they're great for structured extraction tasks